In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import shap
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, cross_val_score, cross_val_predict
from sklearn.metrics import matthews_corrcoef, confusion_matrix, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from sklearn.metrics import recall_score, accuracy_score
from sklearn.impute import KNNImputer, SimpleImputer

In [2]:
random_state = 7
cv_method = StratifiedKFold(n_splits=5)

# Training of tsfresh-selected Training Max 2 years and eGFR 8 times

In [3]:
df = pd.read_csv('/home/jupyter-dchristiadi85/PhD Project 1/data/train_selected_2_8.gz')
df.shape

(4338, 23639)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4338 entries, 0 to 4337
Columns: 23639 entries, id to cat8
dtypes: float64(23617), int64(2), object(20)
memory usage: 782.4+ MB


In [5]:
df['aki_1'].fillna(value=0, inplace=True)
df['aki_2'].fillna(value=0, inplace=True)
df['aki_3'].fillna(value=0, inplace=True)

In [6]:
df['gn'].value_counts()

no                 3892
igan                180
anca                 83
lupus nephritis      45
membranous           32
unspecific           29
fsgs                 27
mcd                  24
mp/mcgn              19
fgn/itg               5
pign                  2
Name: gn, dtype: int64

In [7]:
df.loc[df.gn == 'pign', 'gn'] = 'unspecific'
df.loc[df.gn == 'fgn/itg', 'gn'] = 'unspecific'

In [8]:
percent_missing = df.isnull().sum()*100 / len(df)
missing_df = pd.DataFrame({'column_name': df.columns, 'percent_miss': percent_missing})

In [9]:
missing_df.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
"crp__fft_coefficient__attr_""""""""""""""""imag""""""""""""""""__coeff_70","crp__fft_coefficient__attr_""""""""""""""""imag""""""""""""""...",100.0
"crp__fft_coefficient__attr_""""""""""""""""angle""""""""""""""""__coeff_71","crp__fft_coefficient__attr_""""""""""""""""angle""""""""""""...",100.0
"crp__fft_coefficient__attr_""""""""""""""""angle""""""""""""""""__coeff_73","crp__fft_coefficient__attr_""""""""""""""""angle""""""""""""...",100.0
"crp__fft_coefficient__attr_""""""""""""""""angle""""""""""""""""__coeff_74","crp__fft_coefficient__attr_""""""""""""""""angle""""""""""""...",100.0
"crp__fft_coefficient__attr_""""""""""""""""angle""""""""""""""""__coeff_75","crp__fft_coefficient__attr_""""""""""""""""angle""""""""""""...",100.0
...,...,...
htn,htn,0.0
aki_3,aki_3,0.0
aki_2,aki_2,0.0
aki_1,aki_1,0.0


### Number of patients reaching ESKD within training period 

In [10]:
df['cat2'].value_counts()

non_eskd    3159
eskd        1179
Name: cat2, dtype: int64

In [11]:
df.set_index('id', inplace=True)

In [3]:
def to_category(df):
    cols = df.select_dtypes(include='object').columns
    for col in cols:
        df[col] = df[col].astype('category')
    return df

def drop_missing(df):
    threshold = len(df)*0.6
    df.dropna(axis=1, thresh=threshold, inplace=True)
    return df

def copy_df(df):
    return df.copy()

In [13]:
df_cleaned = (df.pipe(copy_df).pipe(drop_missing).pipe(to_category))
df_cleaned.shape

(4338, 6802)

In [14]:
df_rm_intrain = df_cleaned.query("cat2 == 'non_eskd'")
df_rm_intrain.shape

(3159, 6802)

In [23]:
df_rm_intrain = pd.read_csv('../data/cleaned60_selected_2_8.gz', index_col='id')

In [24]:
df_rm_intrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3159 entries, 3918 to 23864
Columns: 6802 entries, ureacreat_ratio__variance_larger_than_standard_deviation to cat8
dtypes: float64(6781), int64(1), object(20)
memory usage: 164.0+ MB


In [25]:
df_rm_intrain = (df_rm_intrain.pipe(copy_df).pipe(to_category))
df_rm_intrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3159 entries, 3918 to 23864
Columns: 6802 entries, ureacreat_ratio__variance_larger_than_standard_deviation to cat8
dtypes: category(20), float64(6781), int64(1)
memory usage: 163.5 MB


### Number of patients reaching ESKD in at 1y post-training 

In [17]:
df_rm_intrain['cat3'].value_counts()

non_eskd    3038
eskd         121
Name: cat3, dtype: int64

### Number of patients reaching ESKD in 2y post-training

In [18]:
df_rm_intrain['cat4'].value_counts()

non_eskd    2939
eskd         220
Name: cat4, dtype: int64

### Number of patients reaching ESKD in 5y post-training

In [19]:
df_rm_intrain['cat7'].value_counts()

non_eskd    2719
eskd         440
Name: cat7, dtype: int64

In [20]:
percent_missing = df_rm_intrain.isnull().sum()*100 / len(df_rm_intrain)
missing_df_rm_intrain = pd.DataFrame({'column_name': df_rm_intrain.columns, 'percent_miss': percent_missing})

In [21]:
missing_df_rm_intrain.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
egfr__autocorrelation__lag_5,egfr__autocorrelation__lag_5,51.693574
egfr__partial_autocorrelation__lag_2,egfr__partial_autocorrelation__lag_2,51.693574
creatinine__autocorrelation__lag_4,creatinine__autocorrelation__lag_4,51.218740
creatinine__permutation_entropy__dimension_5__tau_1,creatinine__permutation_entropy__dimension_5__...,51.187085
"egfr__agg_linear_trend__attr_""""""""""""""""stderr""""""""""""""""__chunk_len_5__f_agg_""""""""""""""""min""""""""""""""""","egfr__agg_linear_trend__attr_""""""""""""""""stderr""""""...",51.187085
...,...,...
dkd,dkd,0.000000
htn,htn,0.000000
aki_3,aki_3,0.000000
aki_2,aki_2,0.000000


## ESKD Prediction 5 years post-training

In [6]:
dropped_cols = list(df_rm_intrain.columns[-18:])
dropped_cols.remove('age.init')
dropped_cols

['egfr.y',
 'cat0.5',
 'cat1',
 'cat1.5',
 'cat2',
 'cat2.5',
 'cat3',
 'cat3.5',
 'cat4',
 'cat4.5',
 'cat5',
 'cat5.5',
 'cat6',
 'cat6.5',
 'cat7',
 'cat7.5',
 'cat8']

In [23]:
X = df_rm_intrain.drop(dropped_cols,axis=1).copy()
y = df_rm_intrain['cat7']
X.shape, y.shape

((3159, 6785), (3159,))

In [24]:
y_mapped = y.map({'non_eskd':'no', 'eskd':'yes'})
y_mapped

id
3918      no
3921      no
3924      no
3930      no
3934     yes
        ... 
23771     no
23775     no
23780     no
23823     no
23864     no
Name: cat7, Length: 3159, dtype: category
Categories (2, object): ['yes', 'no']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.3, stratify=y_mapped, random_state=random_state)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2211, 6785), (948, 6785), (2211,), (948,))

In [26]:
lab_encode = LabelEncoder()
y_train = lab_encode.fit_transform(y_train)
y_test = lab_encode.transform(y_test)
y_train.shape, y_test.shape

((2211,), (948,))

In [27]:
unique_values, counts = np.unique(y_train, return_counts=True)
ratio = dict(zip(unique_values, counts))
ratio

{0: 1903, 1: 308}

In [28]:
imbalance_ratio = ratio[0] / ratio[1]
imbalance_ratio

6.178571428571429

In [29]:
category_cols = list(X_train.select_dtypes(include='category').columns)
numeric_cols = list(X_train.select_dtypes(include=['int', 'float']).columns)
one_hot = OneHotEncoder(handle_unknown='ignore')
num_imputer = KNNImputer(n_neighbors=5)
len(category_cols), len(numeric_cols)

(4, 6781)

In [30]:
preprocess = ColumnTransformer([('num_imputing', num_imputer, numeric_cols), ('cat_encoder', one_hot, category_cols)], remainder='passthrough')
X_train_processed = preprocess.fit_transform(X_train)
X_train_processed.shape

(2211, 6796)

In [32]:
xgb_class = xgb.XGBClassifier(n_jobs=-1, random_state = random_state, n_estimators=1000, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=imbalance_ratio)
sampler = SMOTETomek(sampling_strategy=0.5, random_state=random_state)
steps = [('resampling', sampler), ('model', xgb_class)]
pipeline = Pipeline(steps=steps)

In [33]:
pipeline.fit(X_train_processed, y_train)

Pipeline(steps=[('resampling',
                 SMOTETomek(random_state=7, sampling_strategy=0.5)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='logloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=1000,
                               n_jobs=-1, num_parallel_tree=1, random_state=7,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=6.178571428571429, subsample=1,
                               tree_method='exact', use_label_encoder=False,
     

In [34]:
y_pred = pipeline.predict(X_test_processed)

In [35]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[779,  37],
       [ 70,  62]])

In [36]:
roc_auc_score(y_test, y_pred)

0.712176916221034

In [37]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.92      0.95      0.94       816
           1       0.63      0.47      0.54       132

    accuracy                           0.89       948
   macro avg       0.77      0.71      0.74       948
weighted avg       0.88      0.89      0.88       948



In [38]:
print(matthews_corrcoef(y_test, y_pred))

0.4803842181799097


In [39]:
importance = pipeline.named_steps['model'].feature_importances_
importance.shape, type(importance)

((6796,), numpy.ndarray)

In [40]:
category_label = list(preprocess.named_transformers_['cat_encoder'].get_feature_names(category_cols))
feature_label = numeric_cols + category_label
len(feature_label)

6796

In [41]:
pd.set_option('display.max_rows', 400)
eval_df = pd.DataFrame({'label': feature_label, 'importance_value': importance})
eval_df.sort_values(by='importance_value', ascending=False).head(100)

,label,importance_value
1884,creatinine__quantile__q_0.3,0.062252
1888,creatinine__quantile__q_0.8,0.024052
3992,urea__quantile__q_0.2,0.018376
4091,"urea__change_quantiles__f_agg_""""""""""""""""var""""""""""...",0.017391
1832,creatinine__maximum,0.015043
671,"albumin__change_quantiles__f_agg_""""""""""""""""var""""...",0.011659
4638,ggt__time_reversal_asymmetry_statistic__lag_3,0.010624
615,albumin__quantile__q_0.3,0.010466
4141,urea__energy_ratio_by_chunks__num_segments_10_...,0.010311
5007,globulin__approximate_entropy__m_2__r_0.7,0.009654


## ESKD Prediction 2 years post-training

In [27]:
X = df_rm_intrain.drop(dropped_cols,axis=1).copy()
y = df_rm_intrain['cat4']
X.shape, y.shape

((3159, 6785), (3159,))

In [28]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3159 entries, 3918 to 23864
Columns: 6785 entries, ureacreat_ratio__variance_larger_than_standard_deviation to age.init
dtypes: category(4), float64(6780), int64(1)
memory usage: 163.5 MB


In [29]:
df_rm_intrain['cat4'].value_counts()

non_eskd    2939
eskd         220
Name: cat4, dtype: int64

In [22]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3159 entries, 0 to 3158
Columns: 6786 entries, id to age.init
dtypes: float64(6780), int64(2), object(4)
memory usage: 163.6+ MB


In [30]:
y_mapped = y.map({'non_eskd':'no', 'eskd':'yes'})
y_mapped

id
3918     no
3921     no
3924     no
3930     no
3934     no
         ..
23771    no
23775    no
23780    no
23823    no
23864    no
Name: cat4, Length: 3159, dtype: category
Categories (2, object): ['yes', 'no']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.3, stratify=y_mapped, random_state=random_state)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2211, 6785), (948, 6785), (2211,), (948,))

In [32]:
lab_encode = LabelEncoder()
y_train = lab_encode.fit_transform(y_train)
y_test = lab_encode.transform(y_test)
y_train.shape, y_test.shape

((2211,), (948,))

In [33]:
unique_values, counts = np.unique(y_train, return_counts=True)
ratio = dict(zip(unique_values, counts))
ratio

{0: 2057, 1: 154}

In [34]:
imbalance_ratio = ratio[0] / ratio[1]
imbalance_ratio

13.357142857142858

In [35]:
category_cols = list(X_train.select_dtypes(include='category').columns)
numeric_cols = list(X_train.select_dtypes(include=['int', 'float']).columns)
one_hot = OneHotEncoder(handle_unknown='ignore')
num_imputer = KNNImputer(n_neighbors=5)
len(category_cols), len(numeric_cols)

(4, 6781)

In [36]:
preprocess = ColumnTransformer([('num_imputing', num_imputer, numeric_cols), ('cat_encoder', one_hot, category_cols)], remainder='passthrough')
X_train_processed = preprocess.fit_transform(X_train)
X_train_processed.shape

(2211, 6796)

In [37]:
X_test_processed = preprocess.transform(X_test)

In [38]:
xgb_class = xgb.XGBClassifier(n_jobs=-1, random_state = random_state, n_estimators=1000, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=imbalance_ratio)
sampler = SMOTETomek(random_state=random_state)
steps = [('resampling', sampler), ('model', xgb_class)]
pipeline = Pipeline(steps=steps)

In [39]:
pipeline.fit(X_train_processed, y_train)

Pipeline(steps=[('resampling', SMOTETomek(random_state=7)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='logloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=1000,
                               n_jobs=-1, num_parallel_tree=1, random_state=7,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=13.357142857142858, subsample=1,
                               tree_method='exact', use_label_encoder=False,
                               validate_para

In [40]:
y_pred = pipeline.predict(X_test_processed)

In [41]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[851,  31],
       [ 45,  21]])

In [42]:
roc_auc_score(y_test, y_pred)

0.6415172129457843

In [43]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96       882
           1       0.40      0.32      0.36        66

    accuracy                           0.92       948
   macro avg       0.68      0.64      0.66       948
weighted avg       0.91      0.92      0.92       948



In [44]:
print(matthews_corrcoef(y_test, y_pred))

0.31636603117366746


In [45]:
importance = pipeline.named_steps['model'].feature_importances_
importance.shape, type(importance)

((6796,), numpy.ndarray)

In [46]:
category_label = list(preprocess.named_transformers_['cat_encoder'].get_feature_names(category_cols))
feature_label = numeric_cols + category_label
len(feature_label)

6796

In [47]:
pd.set_option('display.max_rows', 400)
eval_df = pd.DataFrame({'label': feature_label, 'importance_value': importance})
eval_df.sort_values(by='importance_value', ascending=False).head(100)

,label,importance_value
3998,urea__quantile__q_0.9,0.089570
761,albumin__energy_ratio_by_chunks__num_segments_...,0.027949
1280,calcium__fourier_entropy__bins_2,0.024566
1805,creatinine__abs_energy,0.024125
350,wcc__quantile__q_0.1,0.018936
3994,urea__quantile__q_0.4,0.017513
602,albumin__large_standard_deviation__r_0.45,0.017269
6467,"neutrophils__change_quantiles__f_agg_""""""""""""""""m...",0.015076
1888,creatinine__quantile__q_0.8,0.014025
819,alkphos__first_location_of_minimum,0.013385
